# **Proyek Pengembangan dan Pengoperasian Sistem Machine Learning**

* Nama : Moh. Aflah Azzaky
* Email : aflahazzaki123@gmail.com
* ID Dicoding : aflahazzaky
* Dataset : [Heart Failure Prediction Dataset](https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction)

## Create Folder

In [1]:
import os
# Buat folder 'modules, config, monitoring' jika belum ada
os.makedirs('modules', exist_ok=True)
os.makedirs('config', exist_ok=True)
os.makedirs('monitoring', exist_ok=True)

## Import Library

In [2]:
import os
import pandas as pd
import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
import keras_tuner as kt

from absl import logging
from keras import layers
from tfx.components import (CsvExampleGen, Evaluator, ExampleValidator, Pusher,
                            SchemaGen, StatisticsGen, Trainer, Transform, Tuner)
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import \
    LatestBlessedModelStrategy
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from tfx.proto import example_gen_pb2, pusher_pb2, trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
from typing import Text, Any, Dict, NamedTuple
from keras_tuner.engine import base_tuner
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

## Prepare Modules, Config, & Monitoring

### Set Variable Path

In [3]:
COMPONENTS_MODULE_FILE = 'modules/heart_failure_components.py'
PIPELINE_MODULE_FILE = 'modules/heart_failure_pipeline.py'
TRAINER_MODULE_FILE = 'modules/heart_failure_trainer.py'
TRANSFORM_MODULE_FILE = 'modules/heart_failure_transform.py'
TUNER_MODULE_FILE = 'modules/heart_failure_tuner.py'

CONFIG_PATH = 'config/prometheus.config'
MONITORING_PATH = 'monitoring/prometheus.yml'
MONITORING_DOCKER_PATH = 'monitoring/Dockerfile'

### Create Components

In [ ]:
%%writefile {COMPONENTS_MODULE_FILE}
import os

import tensorflow_model_analysis as tfma
from tfx.components import (CsvExampleGen, Evaluator, ExampleValidator, Pusher,
                            SchemaGen, StatisticsGen, Trainer, Transform,
                            Tuner)
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import \
    LatestBlessedModelStrategy
from tfx.proto import example_gen_pb2, pusher_pb2, trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing


def init_components(args):
    """Initiate tfx pipeline components

    Args:
        args (dict): args that containts some dependencies

    Returns:
        tuple: TFX pipeline components
    """
    output = example_gen_pb2.Output(
        split_config=example_gen_pb2.SplitConfig(splits=[
            example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
            example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2),
        ])
    )

    example_gen = CsvExampleGen(
        input_base=args["data_dir"],
        output_config=output,
    )

    statistics_gen = StatisticsGen(
        examples=example_gen.outputs["examples"],
    )

    schema_gen = SchemaGen(
        statistics=statistics_gen.outputs["statistics"],
    )

    example_validator = ExampleValidator(
        statistics=statistics_gen.outputs["statistics"],
        schema=schema_gen.outputs["schema"],
    )

    transform = Transform(
        examples=example_gen.outputs["examples"],
        schema=schema_gen.outputs["schema"],
        module_file=os.path.abspath(args["transform_module"]),
    )

    tuner = Tuner(
        module_file=os.path.abspath(args["tuner_module"]),
        examples=transform.outputs["transformed_examples"],
        transform_graph=transform.outputs["transform_graph"],
        schema=schema_gen.outputs["schema"],
        train_args=trainer_pb2.TrainArgs(
            splits=["train"],
            num_steps=args["train_steps"],
        ),
        eval_args=trainer_pb2.EvalArgs(
            splits=["eval"],
            num_steps=args["eval_steps"],
        ),
    )

    trainer = Trainer(
        module_file=args["trainer_module"],
        examples=transform.outputs["transformed_examples"],
        transform_graph=transform.outputs["transform_graph"],
        schema=schema_gen.outputs["schema"],
        hyperparameters=tuner.outputs["best_hyperparameters"],
        train_args=trainer_pb2.TrainArgs(
            splits=["train"],
            num_steps=args["train_steps"],
        ),
        eval_args=trainer_pb2.EvalArgs(
            splits=["eval"],
            num_steps=args["eval_steps"]
        ),
    )

    model_resolver = Resolver(
        strategy_class=LatestBlessedModelStrategy,
        model=Channel(type=Model),
        model_blessing=Channel(type=ModelBlessing),
    ).with_id("Latest_blessed_model_resolve")

    eval_config = tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key="HeartDisease")],
        slicing_specs=[
            tfma.SlicingSpec(),
            tfma.SlicingSpec(feature_keys=["Sex"]),
        ],
        metrics_specs=[
            tfma.MetricsSpec(metrics=[
                tfma.MetricConfig(class_name="AUC"),
                tfma.MetricConfig(class_name="Precision"),
                tfma.MetricConfig(class_name="Recall"),
                tfma.MetricConfig(class_name="ExampleCount"),
                tfma.MetricConfig(class_name="TruePositives"),
                tfma.MetricConfig(class_name="FalsePositives"),
                tfma.MetricConfig(class_name="TrueNegatives"),
                tfma.MetricConfig(class_name="FalseNegatives"),
                tfma.MetricConfig(
                    class_name="BinaryAccuracy",
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={"value": .6},
                        ),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={"value": 1e-4},
                        ),
                    ),
                ),
            ]),
        ],
    )

    evaluator = Evaluator(
        examples=example_gen.outputs["examples"],
        model=trainer.outputs["model"],
        baseline_model=model_resolver.outputs["model"],
        eval_config=eval_config,
    )

    pusher = Pusher(
        model=trainer.outputs["model"],
        model_blessing=evaluator.outputs["blessing"],
        push_destination=pusher_pb2.PushDestination(
            filesystem=pusher_pb2.PushDestination.Filesystem(
                base_directory=args["serving_model_dir"],
            ),
        ),
    )

    return (
        example_gen,
        statistics_gen,
        schema_gen,
        example_validator,
        transform,
        tuner,
        trainer,
        model_resolver,
        evaluator,
        pusher,
    )

### Create Pipeline

In [ ]:
%%writefile {PIPELINE_MODULE_FILE}
from typing import Text
from absl import logging
from tfx.orchestration import metadata, pipeline

def init_pipeline(pipeline_root: Text, pipeline_name, metadata_path, components):
    """Initiate tfx pipeline

    Args:
        pipeline_root (Text): a path to th pipeline directory
        pipeline_name (str): pipeline name
        metadata_path (str): a path to the metadata directory
        components (dict): tfx components

    Returns:
        pipeline.Pipeline: pipeline orchestration
    """

    logging.info(f"Pipeline root set to: {pipeline_root}")

    beam_args = [
        "--direct_running_mode=multi_processing",
        "----direct_num_workers=0",
    ]

    return pipeline.Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path,
        ),
        eam_pipeline_args=beam_args,
    )

### Create Trainer

In [ ]:
%%writefile {TRAINER_MODULE_FILE}
import os
import tensorflow as tf
import tensorflow_transform as tft
from keras import layers
from heart_failure_transform import (CATEGORICAL_FEATURES, LABEL_KEY, NUMERICAL_FEATURES,
                       transformed_name)
from heart_failure_tuner import input_fn

def get_serve_tf_examples_fn(model, tf_transform_output):
    """Return a function that parses a serialized tf.Example"""

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None], dtype=tf.string, name="examples"),
    ])
    def serve_tf_examples_fn(serialized_tf_examples):
        """Return the output to be used in the serving signature."""

        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)

        parsed_features = tf.io.parse_example(
            serialized_tf_examples, feature_spec,
        )

        transformed_features = model.tft_layer(parsed_features)
        outputs = model(transformed_features)

        return {"outputs": outputs}

    return serve_tf_examples_fn


def get_model(hyperparameters):
    """This model defines a keras Model

    Args:
        hyperparameters (kt.HyperParameters): object that contains best hyperparameters
        from tuner

    Returns:
        tf.keras.Model: model as a Keras object
    """

    input_features = []

    for key, dim in CATEGORICAL_FEATURES.items():
        input_features.append(
            layers.Input(shape=(dim+1,), name=transformed_name(key))
        )

    for feature in NUMERICAL_FEATURES:
        input_features.append(
            layers.Input(shape=(1,), name=transformed_name(feature))
        )

    concatenate = layers.concatenate(input_features)
    deep = layers.Dense(
        hyperparameters["dense_unit"], activation=tf.nn.relu)(concatenate)

    for _ in range(hyperparameters["num_hidden_layers"]):
        deep = layers.Dense(
            hyperparameters["dense_unit"], activation=tf.nn.relu)(deep)
        deep = layers.Dropout(hyperparameters["dropout_rate"])(deep)

    outputs = layers.Dense(1, activation=tf.nn.sigmoid)(deep)

    model = tf.keras.Model(inputs=input_features, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hyperparameters["learning_rate"]),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=["binary_accuracy"],
    )

    model.summary()

    return model


def run_fn(fn_args):
    """Train the model based on given args

    Args:
        fn_args (FnArgs): Holds args used to train the model as name/value pairs.
    """

    hyperparameters = fn_args.hyperparameters["values"]
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), "logs")

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_set = input_fn(fn_args.train_files, tf_transform_output)
    eval_set = input_fn(fn_args.eval_files, tf_transform_output)

    model = get_model(hyperparameters)

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        update_freq="batch"
    )

    early_stop_callbacks = tf.keras.callbacks.EarlyStopping(
        monitor="val_binary_accuracy",
        mode="max",
        verbose=1,
        patience=10,
    )

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        fn_args.serving_model_dir,
        monitor="val_binary_accuracy",
        mode="max",
        verbose=1,
        save_best_only=True,
    )

    callbacks = [
        tensorboard_callback,
        early_stop_callbacks,
        model_checkpoint_callback
    ]

    model.fit(
        x=train_set,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_set,
        validation_steps=fn_args.eval_steps,
        callbacks=callbacks,
        epochs=hyperparameters["tuner/initial_epoch"],
        verbose=1,
    )

    signatures = {
        "serving_default": get_serve_tf_examples_fn(
            model, tf_transform_output,
        )
    }

    model.save(
        fn_args.serving_model_dir,
        save_format="tf",
        signatures=signatures,
    )

### Create Transform

In [ ]:
%%writefile {TRANSFORM_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft

CATEGORICAL_FEATURES = {
    "Sex": 2,
    "ChestPainType": 4,
    "RestingECG": 3,
    "ExerciseAngina": 2,
    "ST_Slope": 3,
}

NUMERICAL_FEATURES = [
    "Age",
    "RestingBP",
    "Cholesterol",
    "FastingBS",
    "MaxHR",
    "Oldpeak",
]

LABEL_KEY = "HeartDisease"


def transformed_name(key):
    """Transform feature key

    Args:
        key (str): the key to be transformed

    Returns:
        str: transformed key
    """

    return f"{key}_xf"


def convert_num_to_one_hot(label_tensor, num_labels=2):
    """Convert a label (0 or 1) into a one-hot vector

    Args:
        label_tensor (int): label tensor (0 or 1)
        num_labels (int, optional): num of label. Defaults to 2.

    Returns:
        tf.Tensor: label tensor
    """

    one_hot_tensor = tf.one_hot(label_tensor, num_labels)
    return tf.reshape(one_hot_tensor, [-1, num_labels])


def replace_nan(tensor):
    """Replace nan value with zero number

    Args:
        tensor (list): list data with na data that want to replace

    Returns:
        list with replaced nan value
    """
    tensor = tf.cast(tensor, tf.float64)
    return tf.where(
        tf.math.is_nan(tensor),
        tft.mean(tensor),
        tensor
    )


def preprocessing_fn(inputs):
    """Preprocess input features into transformed features

    Args:
        inputs (dict): map from feature keys to raw features

    Returns:
        dict: map from features keys to transformed features
    """

    outputs = {}

    for keys, values in CATEGORICAL_FEATURES.items():
        int_value = tft.compute_and_apply_vocabulary(
            inputs[keys], top_k=values+1)
        outputs[transformed_name(keys)] = convert_num_to_one_hot(
            int_value, num_labels=values+1)

    for feature in NUMERICAL_FEATURES:
        inputs[feature] = replace_nan(inputs[feature])
        outputs[transformed_name(feature)] = tft.scale_to_0_1(inputs[feature])

    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)

    return outputs

### Create Tuner

In [ ]:
%%writefile {TUNER_MODULE_FILE}
from typing import Any, Dict, NamedTuple, Text

import keras_tuner as kt
import tensorflow as tf
import tensorflow_transform as tft
from keras import layers
from keras_tuner.engine import base_tuner
from heart_failure_transform import (CATEGORICAL_FEATURES, LABEL_KEY, NUMERICAL_FEATURES,
                       transformed_name)

NUM_EPOCHS = 5

TunerFnResult = NamedTuple("TunerFnResult", [
    ("tuner", base_tuner.BaseTuner),
    ("fit_kwargs", Dict[Text, Any]),
])

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_binary_accuracy",
    mode="max",
    verbose=1,
    patience=10,
)


def gzip_reader_fn(filenames):
    """Loads compression data

    Args:
        filenames (str): a path to the data directory

    Returns:
        TfRecord: Compressed data
    """

    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")


def input_fn(file_pattern, tf_transform_output, batch_size=64):
    """Generated features and labels for tuning/training

    Args:
        file_pattern: input tf_record file pattern
        tf_transform_output: a TFTransformOutput
        batch_size: representing the number of consecutive elements of
        returned dataset to combine in a single batch. Defaults to 64.

    Returns:
        a dataset that contains (featurs, indices) tuple where features
        is a dictionary of Tensors, and indices is a single Tensor of
        label indices
    """

    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        label_key=transformed_name(LABEL_KEY)
    )

    return dataset


def get_model_tuner(hyperparameters):
    """This function defines a keras Model

    Args:
        hyperparameters (kt.HyperParameters): object to setting hyperparameters

    Returns:
        tf.keras.Model: model as a Keras object
    """

    num_hidden_layers = hyperparameters.Choice(
        "num_hidden_layers",
        values=[1, 2, 3],
    )
    dense_unit = hyperparameters.Int(
        "dense_unit",
        min_value=16,
        max_value=256,
        step=32,
    )
    dropout_rate = hyperparameters.Float(
        "dropout_rate",
        min_value=0.1,
        max_value=0.9,
        step=0.1,
    )
    learning_rate = hyperparameters.Choice(
        "learning_rate",
        values=[1e-2, 1e-3, 1e-4]
    )

    input_features = []

    for key, dim in CATEGORICAL_FEATURES.items():
        input_features.append(
            layers.Input(shape=(dim+1,), name=transformed_name(key))
        )

    for feature in NUMERICAL_FEATURES:
        input_features.append(
            layers.Input(shape=(1,), name=transformed_name(feature))
        )

    concatenate = layers.concatenate(input_features)
    deep = layers.Dense(dense_unit, activation=tf.nn.relu)(concatenate)

    for _ in range(num_hidden_layers):
        deep = layers.Dense(dense_unit, activation=tf.nn.relu)(deep)
        deep = layers.Dropout(dropout_rate)(deep)

    outputs = layers.Dense(1, activation=tf.nn.sigmoid)(deep)

    model = tf.keras.Model(inputs=input_features, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=["binary_accuracy"],
    )

    model.summary()

    return model


def tuner_fn(fn_args):
    """Tuning the model to get the best hyperparameters based on given args

    Args:
        fn_args (FnArgs): Holds args used to train the model as name/value pair

    Returns:
        TunerFnResult (NamedTuple): object to run model tuner
    """

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_set = input_fn(fn_args.train_files[0], tf_transform_output,)
    eval_set = input_fn(fn_args.eval_files[0], tf_transform_output,)

    tuner = kt.Hyperband(
        hypermodel=get_model_tuner,
        objective=kt.Objective(
            "val_loss",
            direction="min",
        ),
        max_epochs=NUM_EPOCHS,
        factor=3,
        directory=fn_args.working_dir,
        project_name="kt_hyperband",
    )

    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            "x": train_set,
            "validation_data": eval_set,
            "steps_per_epoch": fn_args.train_steps,
            "validation_steps": fn_args.eval_steps,
            "callbacks": [early_stop]
        },
    )

### Create Config

In [ ]:
%%writefile {CONFIG_PATH}
prometheus_config {
   enable: true,
   path: "/monitoring/prometheus/metrics"
}

### Create Monitoring Prometheus

In [ ]:
%%writefile {MONITORING_PATH}
global:
  scrape_interval: 15s
  evaluation_interval: 15s
  external_labels:
    monitor: "tf-serving-monitor"

scrape_configs:
  - job_name: "prometheus"
    scrape_interval: 5s
    metrics_path: /monitoring/prometheus/metrics
    static_configs:
      - targets: ["hf-pred.herokuapp.com"]


### Create Monitoring Dockerfile

In [ ]:
%%writefile {MONITORING_DOCKER_PATH}
FROM prom/prometheus:latest

COPY prometheus.yml /etc/prometheus/prometheus.yml

## Prepare Dataset

### Load Dataset

In [ ]:
url_dataset = 'data/heart.csv'
df = pd.read_csv(url_dataset)
df.head(1)

### Description Dataset

In [ ]:
df.info()

### Cleaning Dataset

In [ ]:
RestingBP = (df.RestingBP == 0).sum()
Cholesterol = (df.Cholesterol == 0).sum()

print(f'Jumlah nilai 0 pada RestingBP: {RestingBP}')
print(f'Jumlah nilai 0 pada Cholesterol: {Cholesterol}')

In [ ]:
df.drop(df.loc[(df['RestingBP']==0)].index, inplace=True)
df.drop(df.loc[(df['Cholesterol']==0)].index, inplace=True)
df.shape

In [16]:
data_path = 'data_cleaned'

if not os.path.exists(data_path):
    os.makedirs(data_path)

df.to_csv(os.path.join(data_path, 'heart_failure_cleaned.csv'), index=False)

## Set Variable

In [17]:
from modules import heart_failure_components, heart_failure_pipeline

PIPELINE_NAME = 'aflahazzaky-pipeline'

# Pipeline inputs
DATA_ROOT = 'data_cleaned'
TRAINER_MODULE_FILE = 'modules/heart_failure_trainer.py'
TRANSFORM_MODULE_FILE = 'modules/heart_failure_transform.py'
TUNER_MODULE_FILE = 'modules/heart_failure_tuner.py'

# Pipeline outputs
OUTPUT_ROOT = 'outputs'

serving_model_dir = os.path.join(OUTPUT_ROOT, 'serving_model')
pipeline_root = os.path.join(OUTPUT_ROOT, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, 'metadata.sqlite')

## Run Pipeline

In [ ]:
components_args = {
    "data_dir": DATA_ROOT,
    "trainer_module": TRAINER_MODULE_FILE,
    "tuner_module": TUNER_MODULE_FILE,
    "transform_module": TRANSFORM_MODULE_FILE,
    "train_steps": 1000,
    "eval_steps": 800,
    "serving_model_dir": serving_model_dir,
}

components = heart_failure_components.init_components(components_args)

pipeline = heart_failure_pipeline.init_pipeline(
    pipeline_root, 
    PIPELINE_NAME, 
    metadata_path, 
    components
)
BeamDagRunner().run(pipeline)